# Setup

In [1]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
from models import (BasicCNN, AutoencoderCNN, ECACNN, AutoencoderECACNN,
                    ECASpatialCNN, DeeperCNN, LinearAutoencoderECACNN,
                    CBAMCNN, LinearAutoencoderCNN)


## Loading the Models

In [3]:
cnn_model = BasicCNN()
eca_model = ECACNN()
cbam_model = CBAMCNN()
deeper_cnn_model = DeeperCNN()
autoencoder_model = AutoencoderCNN()
autoencoder_eca_model = AutoencoderECACNN()
linear_autoencoder_model = LinearAutoencoderCNN()
linear_autoencoder_eca_model = LinearAutoencoderECACNN()
eca_spatial_model = ECASpatialCNN()


models = {
    'cnn': cnn_model,
    'eca': eca_model,
    'cbam': cbam_model,
    'deeper_cnn': deeper_cnn_model,
    'autoencoder': autoencoder_model,
    'autoencoder_eca': autoencoder_eca_model,
    'linear_autoencoder': linear_autoencoder_model,
    'linear_autoencoder_eca': linear_autoencoder_eca_model,
    'eca_spatial': eca_spatial_model,
    }

## Loading Results Data

In [4]:
with open('results.pkl', 'rb') as f:
    results = pickle.load(f)



---



# Results analysis

In [5]:
average_test_accuracies = {}
best_test_accuracies = {}
average_test_losses = {}
average_best_valid_accuracies = {}
std_test_accuracies = {}


for name, model in results.items():
    test_accuracies = [model[run]['test_accuracy'] for run in range(len(model))]
    test_losses = [model[run]['test_loss'] for run in range(len(model))]
    best_valid_accuracies = [np.max(model[run]['logging_dict']['accuracy']['validation']) for run in range(len(model))]

    average_test_accuracies[name] = np.average(test_accuracies)
    average_test_losses[name] = np.average(test_losses)
    average_best_valid_accuracies[name] = np.average(best_valid_accuracies)
    best_test_accuracies[name] = np.max(test_accuracies)
    std_test_accuracies[name] = np.std(test_accuracies)


average_test_accuracies, std_test_accuracies

({'cnn': 0.6667511372626582,
  'eca': 0.6674557456487342,
  'cbam': 0.6418364319620253,
  'autoencoder': 0.6746439873417722,
  'autoencoder_eca': 0.6652862935126582,
  'linear_autoencoder_eca': 0.6087754647943038,
  'eca_spatial': 0.6685682852056962,
  'deeper_cnn': 0.6645075158227849,
  'linear_autoencoder': 0.6023289161392404},
 {'cnn': 0.018893083368881534,
  'eca': 0.023452770114145892,
  'cbam': 0.025581481626441922,
  'autoencoder': 0.01988106309039409,
  'autoencoder_eca': 0.018313657079510656,
  'linear_autoencoder_eca': 0.025539537252491733,
  'eca_spatial': 0.013558224866598865,
  'deeper_cnn': 0.025871021497234928,
  'linear_autoencoder': 0.0392586111388186})

In [180]:
def make_plots(logging_dict, model_name, avg_train=True):
    epoch_ends = np.cumsum(logging_dict['batches_per_epoch'])

    def get_avg_per_epoch(batch_data):
        result = [None,]
        for i in range(len(epoch_ends) - 1):
            result.append(np.average(batch_data[epoch_ends[i]:epoch_ends[i + 1]]))
        return result

    fig, axes = plt.subplots(1, 2, figsize=(10, 3))
    metrics = ('loss', 'accuracy')
    for metric, ax in zip(metrics, axes.ravel()):
        if avg_train:
            ax.plot(get_avg_per_epoch(logging_dict[metric]['train']), '.-', label='training set')
            ax.plot(logging_dict[metric]['validation'], '.-', label='validation set')
            ax.set(title=metric, xlabel='epoch', xticks=np.arange(len(epoch_ends)))
        else:
            ax.plot(logging_dict[metric]['train'],'.-', label='training set')
            ax.plot(epoch_ends, logging_dict[metric]['validation'],'.-', label='validation set')
            ax.set(title=metric, xlabel='batch')

    handles, labels = ax.get_legend_handles_labels()
    plt.figlegend(handles=handles, labels=labels, loc='upper center', bbox_to_anchor=(0.5, 0), ncol=2)
    plt.suptitle(model_name)
    plt.tight_layout()
    plt.show()

---

# Parameter Count

In [6]:
def count_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [7]:
for name, model in models.items():
    print(name, f'{count_params(model):,}')

cnn 2,625,866
eca 2,625,878
cbam 2,628,830
deeper_cnn 3,512,138
autoencoder 2,918,570
autoencoder_eca 2,918,582
linear_autoencoder 2,722,260
linear_autoencoder_eca 2,722,272
eca_spatial 2,626,072
